## Ejercicio 2:

Para responder a la primera pregunta, primero definimos 2 cadenas:

In [75]:
cadena_1="Hola"

"Hola"

In [76]:
cadena_2="Mundo"

"Mundo"

Y ahora tratamos de sumarlas:

In [77]:
cadena_1+cadena_2

LoadError: `+` has no method matching +(::ASCIIString, ::ASCIIString)
while loading In[77], in expression starting on line 1

Observamos que no es posible ya que + no sabe que hacer si ambos argumentos son de tipo "ASCIIString".

Ahora para la pregunta 2, multiplicaremos a nuestras cadenas:

In [78]:
cadena_1*cadena_2

"HolaMundo"

Vemos que multiplicar cadenas si es posible y es equivalente a concatenarlas.

Para la pregunta 3, sabemos que podemos definir un nuevo método de + que al recibir como argumento a dos cadenas las concatene (como en Python), aunque en realidad podemos hacer que + realizara cualquier operación deseada sobre las dos cadenas.
Por ejemplo, podríamos hacer que al sumar dos cadenas, + aparte de concatenarlas coloque un espacio entre ellas:

In [92]:
+(x::ASCIIString,y::ASCIIString)=x*" "*y #Definimos un nuevo método a + cuando ambos argumentos son cadenas.

+ (generic function with 123 methods)

Ya que definimos el nuevo método, vamos a sumar nuestras cadenas:

In [91]:
cadena_1+cadena_2

"Hola Mundo"

Y efectívamente, obtuvimos la cadena esperada.

## Ejercicio 3: Diferenciación automática

Primero invocamos a nuestro módulo AutDiff.jl:

In [61]:
using AutDiff

Probamos si `dual_var()` en `x0` nos regresa al Dual esperado:

In [ ]:
dual_var(1.0)  #Probando con un número real.

In [ ]:
dual_var(π)  #Probando con un número irracional.

In [ ]:
dual_var(5) #Probando con un número entero

Vemos que `dual_var()` funciona como esperábamos, y ahora probaremos que nuestra función `dual_cte()` haga lo esperado (nos debe de regresar el dual de un número):

In [ ]:
arregloprueba=[dual_cte(π),dual_cte(2.0),dual_cte(-10)]  #Probamos dual_cte() con un real, irracional y un entero.

Efectivamente, también hace lo esperado.
Definimos la siguiente función:

$$f(x)=2x+log(x)-x^3$$

De la cual sabemos que su primera derivada es:

$$f\prime(x)=2+\frac{1}{x} -3x^2$$

Por lo que si $x_0 =2$, esperamos que $f\prime(2)=-9.5$.
Entonces, usando Duales calcularemos $f\prime(2)$:

In [67]:
x0=dual_var(2.0) #Creamos un dual que corresponde a x0
f(x)=dual_cte(2.0)x+log(x)-x^3 #Definimos f(x) pero sustituyendo dual_cte() en las constantes.
f(x0)  #Evaluamos f(x) en x0

Dual{Float64}(-3.3068528194400546,-9.5)

Vemos que el valor obtenido es exactamente el mismo al esperado.
Ahora vamos a evaluar $f(x)$ en varios `x0` para ver si el comportamiento persiste:

In [68]:
vectorx0=[dual_var(i) for i in 1:5.0:100]  #Creamos un vector con varios valores de x0

20-element Array{Dual{Float64},1}:
  Dual{Float64}(1.0,1.0)
  Dual{Float64}(6.0,1.0)
 Dual{Float64}(11.0,1.0)
 Dual{Float64}(16.0,1.0)
 Dual{Float64}(21.0,1.0)
 Dual{Float64}(26.0,1.0)
 Dual{Float64}(31.0,1.0)
 Dual{Float64}(36.0,1.0)
 Dual{Float64}(41.0,1.0)
 Dual{Float64}(46.0,1.0)
 Dual{Float64}(51.0,1.0)
 Dual{Float64}(56.0,1.0)
 Dual{Float64}(61.0,1.0)
 Dual{Float64}(66.0,1.0)
 Dual{Float64}(71.0,1.0)
 Dual{Float64}(76.0,1.0)
 Dual{Float64}(81.0,1.0)
 Dual{Float64}(86.0,1.0)
 Dual{Float64}(91.0,1.0)
 Dual{Float64}(96.0,1.0)

In [69]:
df(x)=2+(1/x)-3x^2 #Definimos la derivada de f(x)

df (generic function with 1 method)

In [70]:
res1=Array(Float64,0)  #Creamos un vector vacío
for j in vectorx0      #Evaluamos g(x) en cada x0 y luego anexamos a nuestro vector vacío la segunda entrada del Dual resultante.
    m=f(j).fp
    push!(res1,m)
end

In [72]:
res2=Array(Float64,0)  #Creamos otro vector vacío
for n in vectorx0      #Evaluamos dg(x) en cada x0 y anexamos la imagen al vector res2
    push!(res2,df(n.f))
end

In [74]:
[res1 res2]  #Comparamos las derivadas obtenidas vs las esperadas.

20x2 Array{Float64,2}:
      0.0         0.0  
   -105.833    -105.833
   -360.909    -360.909
   -765.938    -765.938
  -1320.95    -1320.95 
  -2025.96    -2025.96 
  -2880.97    -2880.97 
  -3885.97    -3885.97 
  -5040.98    -5040.98 
  -6345.98    -6345.98 
  -7800.98    -7800.98 
  -9405.98    -9405.98 
 -11161.0    -11161.0  
 -13066.0    -13066.0  
 -15121.0    -15121.0  
 -17326.0    -17326.0  
 -19681.0    -19681.0  
 -22186.0    -22186.0  
 -24841.0    -24841.0  
 -27646.0    -27646.0  

Vemos que para todos los `x0`, se obtuvieron las derivadas exactas usando Duales.
Ahora probaremos con casos más complicados en donde la regla de la cadena se tiene que usar más de una vez.

Definimos las siguientes funciones y sus respectivas derivadas:

In [3]:
g(x)=exp(sin(cos(x^3))) #Definimos g(x)

g (generic function with 1 method)

In [18]:
dg(x)=(-3exp(sin(cos(x^3))))*((x^2)cos(cos(x^3))sin(x^3)) #Definimos a la derivada de g(x)

dg (generic function with 1 method)

In [49]:
h(x)=log(x^2)/exp(x^3)  #Definimos h(x)

h (generic function with 1 method)

In [50]:
dh(x)=(exp(-x^3)*(2-(3x^3)log(x^2)))/x #Definimos a la derivada de h(x)

dh (generic function with 1 method)

Creamos un nuevo vector de $x_0$ para evaluar a $g(x)$ (Cuidamos que los números estén entre 0 y 2 ya que la derivada o no está definida, o bien, crece muy rápido): 

In [14]:
vector2x0=[dual_var(i) for i in -100.0:15.0:100.0]  #Creamos un vector con varios valores de x0

14-element Array{Dual{Float64},1}:
 Dual{Float64}(-100.0,1.0)
  Dual{Float64}(-85.0,1.0)
  Dual{Float64}(-70.0,1.0)
  Dual{Float64}(-55.0,1.0)
  Dual{Float64}(-40.0,1.0)
  Dual{Float64}(-25.0,1.0)
  Dual{Float64}(-10.0,1.0)
    Dual{Float64}(5.0,1.0)
   Dual{Float64}(20.0,1.0)
   Dual{Float64}(35.0,1.0)
   Dual{Float64}(50.0,1.0)
   Dual{Float64}(65.0,1.0)
   Dual{Float64}(80.0,1.0)
   Dual{Float64}(95.0,1.0)

Ahora evaluaremos a $g(x)$ y a su derivada en todos los valores x0 de nuestro vector:

In [15]:
res3=Array(Float64,0)  #Creamos un vector vacío
for j in vector2x0      #Evaluamos g(x) en cada x0 y luego anexamos a nuestro vector vacío la segunda entrada del Dual resultante.
    m=g(j).fp
    push!(res3,m)
end

In [19]:
res4=Array(Float64,0)  #Creamos otro vector vacío
for n in vector2x0      #Evaluamos dg(x) en cada x0 y anexamos la imagen al vector res2
    z=n.f
    push!(res4,dg(z))
end

In [20]:
[res3 res4]

14x2 Array{Float64,2}:
 -13921.5     -13921.5   
   5078.6       5078.6   
  16923.5      16923.5   
   1688.98      1688.98  
  -3343.48     -3343.48  
  -2284.65     -2284.65  
    357.68       357.68  
     66.2141      66.2141
  -1275.84     -1275.84  
   4064.39      4064.39  
  -2313.91     -2313.91  
   7734.98      7734.98  
  -9290.67     -9290.67  
  -1254.83     -1254.83  

Observamos que la derivada obtenida por medio de Duales es nuevamente idéntica a la esperada.
Ahora creamos otro vector de $x_0$ para evaluar a $h(x)$ y su derivada:

In [57]:
vector3x0=[dual_var(i) for i in -1.0:0.05:1.0]  #Creamos un vector con varios valores de x0

41-element Array{Dual{Float64},1}:
  Dual{Float64}(-1.0,1.0)
 Dual{Float64}(-0.95,1.0)
  Dual{Float64}(-0.9,1.0)
 Dual{Float64}(-0.85,1.0)
  Dual{Float64}(-0.8,1.0)
 Dual{Float64}(-0.75,1.0)
  Dual{Float64}(-0.7,1.0)
 Dual{Float64}(-0.65,1.0)
  Dual{Float64}(-0.6,1.0)
 Dual{Float64}(-0.55,1.0)
  Dual{Float64}(-0.5,1.0)
 Dual{Float64}(-0.45,1.0)
  Dual{Float64}(-0.4,1.0)
                        ⋮
  Dual{Float64}(0.45,1.0)
   Dual{Float64}(0.5,1.0)
  Dual{Float64}(0.55,1.0)
   Dual{Float64}(0.6,1.0)
  Dual{Float64}(0.65,1.0)
   Dual{Float64}(0.7,1.0)
  Dual{Float64}(0.75,1.0)
   Dual{Float64}(0.8,1.0)
  Dual{Float64}(0.85,1.0)
   Dual{Float64}(0.9,1.0)
  Dual{Float64}(0.95,1.0)
   Dual{Float64}(1.0,1.0)

In [58]:
res5=Array(Float64,0)  #Creamos un vector vacío
for j in vector3x0      #Evaluamos g(x) en cada x0 y luego anexamos a nuestro vector vacío la segunda entrada del Dual resultante.
    m=h(j).fp
    push!(res5,m)
end

In [59]:
res6=Array(Float64,0)  #Creamos otro vector vacío
for n in vector3x0     #Evaluamos dg(x) en cada x0 y anexamos la imagen al vector res6
    push!(res6,dh(n.f))
end

In [60]:
[res5 res6]

41x2 Array{Float64,2}:
 -5.43656   -5.43656 
 -4.30738   -4.30738 
 -3.54519   -3.54519 
 -3.04635   -3.04635 
 -2.74177   -2.74177 
 -2.58569   -2.58569 
 -2.54851   -2.54851 
 -2.61219   -2.61219 
 -2.7676    -2.7676  
 -3.01311   -3.01311 
 -3.35444   -3.35444 
 -3.80572   -3.80572 
 -4.39269   -4.39269 
  ⋮                  
  4.94304    4.94304 
  4.44754    4.44754 
  3.99778    3.99778 
  3.57482    3.57482 
  3.16781    3.16781 
  2.77168    2.77168 
  2.38559    2.38559 
  2.01176    2.01176 
  1.65444    1.65444 
  1.31899    1.31899 
  1.01105    1.01105 
  0.735759   0.735759

Y de nuevo, observamos que la derivada de $h(x)$ calculada usando Duales es exacta.